In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

county_population_df = pd.read_excel("resources/CA Gov Department of Finance Population Estimates.xlsx", sheet_name="E-1 CountyState2022",header=1)
county_income_df = pd.read_csv("resources/income-limits-2020.csv")
county_unemployment_df = pd.read_csv("resources/20aacou.csv",header=1)

In [2]:
county_population_df.head(5)

,California,39303157,39185605,-0.3
0,Alameda,1662370,1651979,-0.625071
1,Alpine,1195,1200,0.418410
2,Amador,40287,40297,0.024822
3,Butte,206640,201608,-2.435153
4,Calaveras,45250,45049,-0.444199


In [3]:
county_income_df.head(10)

,Number of Persons in Household:,Unnamed: 1,1,2,3,4,5,6,7,8
0,NaN,Income Category,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Extremely Low,27450.0,31350.0,35250.0,39150.0,42300.0,45450.0,48550.0,51700.0
2,Alameda County,Very Low Income,45700.0,52200.0,58750.0,65250.0,70500.0,75700.0,80950.0,86150.0
3,Area Median Income:,Low Income,73100.0,83550.0,94000.0,104400.0,112800.0,121150.0,129500.0,137850.0
4,"$119,200",Median Income,83450.0,95350.0,107300.0,119200.0,128750.0,138250.0,147800.0,157350.0
5,NaN,Moderate Income,100150.0,114450.0,128750.0,143050.0,154500.0,165950.0,177400.0,188850.0
6,NaN,Extremely Low,18150.0,20750.0,23350.0,26200.0,30680.0,35160.0,39640.0,44120.0
7,Alpine County,Very Low Income,30250.0,34600.0,38900.0,43200.0,46700.0,50150.0,53600.0,57050.0
8,Area Median Income:,Low Income,46600.0,53250.0,59900.0,66550.0,71900.0,77200.0,82550.0,87850.0
9,"$94,900",Median Income,66450.0,75900.0,85400.0,94900.0,102500.0,110100.0,117700.0,125250.0


In [4]:
county_unemployment_df.head(5)

,STATE TOTAL,---,"18,931,100","16,996,700","1,934,500",10.20%
0,ALAMEDA,20,"816,800","743,200","73,600",9.00%
1,ALPINE,49,520,460,60,11.70%
2,AMADOR,29,"14,450","13,080","1,370",9.50%
3,BUTTE,28,"93,100","84,300","8,800",9.40%
4,CALAVERAS,7,"21,320","19,640","1,680",7.90%


In [5]:
income = []
county = []
county_income_df.dropna(inplace=True)
county_income_df["Number of Persons in Household:"].head(5)
for x in county_income_df["Number of Persons in Household:"]:
    if "County" in x:
        county.append(x.split()[0])
    if "$" in x:
        income.append(x)
clean_county_income_df = pd.DataFrame({"county": county, "median_income": income})
clean_county_income_df.head(5)

,county,median_income
0,Alameda,"$119,200"
1,Alpine,"$94,900"
2,Amador,"$78,700"
3,Butte,"$70,700"
4,Calaveras,"$80,400"


In [6]:
clean_county_population_df = county_population_df[["California", 39303157]]
clean_county_population_df.rename(columns={"California":"county", 39303157:"total_population"},inplace=True)
clean_county_population_df.head(5)

,county,total_population
0,Alameda,1662370
1,Alpine,1195
2,Amador,40287
3,Butte,206640
4,Calaveras,45250


In [7]:
clean_county_unemployment_df = county_unemployment_df[["STATE TOTAL", "16,996,700", "1,934,500", "10.20%"]]
clean_county_unemployment_df.rename(columns={"STATE TOTAL": "county", "16,996,700": "employment", "1,934,500": "unemployment", "10.20%": "rate"}, inplace=True)
clean_county_unemployment_df["county"] = clean_county_unemployment_df["county"].str.title()
clean_county_unemployment_df.head(5)

,county,employment,unemployment,rate
0,Alameda,"743,200","73,600",9.00%
1,Alpine,460,60,11.70%
2,Amador,"13,080","1,370",9.50%
3,Butte,"84,300","8,800",9.40%
4,Calaveras,"19,640","1,680",7.90%


In [8]:
from sqlalchemy import create_engine
from config import password

In [9]:
engine = create_engine(f'postgresql://{password}')

In [10]:
clean_county_income_df.to_sql(name='income', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from income', con=engine).head()

,county,median_income
0,Alameda,"$119,200"
1,Alpine,"$94,900"
2,Amador,"$78,700"
3,Butte,"$70,700"
4,Calaveras,"$80,400"


In [11]:
clean_county_population_df.to_sql(name='population', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from population', con=engine).head()

,county,total_population
0,Alameda,1662370
1,Alpine,1195
2,Amador,40287
3,Butte,206640
4,Calaveras,45250


In [12]:
clean_county_unemployment_df.to_sql(name='unemployment', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from unemployment', con=engine).head()

,county,employment,unemployment,rate
0,Alameda,"743,200","73,600",9.00%
1,Alpine,460,60,11.70%
2,Amador,"13,080","1,370",9.50%
3,Butte,"84,300","8,800",9.40%
4,Calaveras,"19,640","1,680",7.90%


In [ ]:
clean_county_income_df.to_excel("income.xlsx")
clean_county_population_df.to_excel("population.xlsx")
clean_county_unemployment_df.to_excel("unemployment.xlsx")